# Analysis of Predictions Produced with Chunks and RNNModel by Darts

This script analyzes all pickle files in `./data/darts/{n_chunks}_chunks/`, starting with `confusion_matrix`, i.e. all model-level and all chunk-level matrices. At the moment, the paths are adapted for local execution.

## Analysis of Model-level Matrices

### Extract All Generated Model-level Matrices

In [ ]:
from IPython.display import display

import os
import pandas as pd
import pickle5 as pickle

# Define number of chunks taken to adjust
n_chunks = 1000

# Define path to all model-level matrices produced by prediction
path_to_model_matrices = f'../../data/darts/{n_chunks}_chunks'

# Collect pickle file names of model-level matrices
model_matrix_filenames = list()

for file in os.listdir(path_to_model_matrices):
    if os.path.isfile(os.path.join(path_to_model_matrices, file)) and \
            file.startswith('confusion_matrix_models') and file.endswith('.pickle'):
        model_matrix_filenames.append(file)

# Concat all found matrices into result matrix
result_matrix_models = pd.DataFrame(columns=['ID', 'PARAMETER', 'MODEL', 'ENDOGENOUS', 'EXOGENOUS', 'FIRST_FORECAST',
                                             'ALARM_TYPE', 'FP', 'TP', 'FN', 'TN', 'N_CHUNKS', 'N_ITERATIONS'])

for filename in model_matrix_filenames:
    # Read file
    current_matrix_f = open(f'{path_to_model_matrices}/{filename}', 'rb')
    current_matrix = pickle.load(current_matrix_f)
    current_matrix_f.close()

    # Append current matrix to result matrix
    result_matrix_models = pd.concat([result_matrix_models, current_matrix])

# Sort result matrix for better readability
result_matrix_models.sort_values(by=['ID'], inplace=True)

# Reset index
result_matrix_models.reset_index(inplace=True, drop=True)

# Show result matrix per parameter
display(result_matrix_models[result_matrix_models['PARAMETER'] == 'HR'])
display(result_matrix_models[result_matrix_models['PARAMETER'] == 'BP'])
display(result_matrix_models[result_matrix_models['PARAMETER'] == 'O2'])

### Add Accuracy Metrics and Save as Parquet Files (Normal + Scaled)

In [ ]:
# Calculate metrics (see https://en.wikipedia.org/wiki/Sensitivity_and_specificity for more information)
result_matrix_models['FPR'] = result_matrix_models['FP'] / (result_matrix_models['FP'] + result_matrix_models['TN'])
result_matrix_models['TPR'] = result_matrix_models['TP'] / (result_matrix_models['TP'] + result_matrix_models['FN'])
result_matrix_models['FNR'] = result_matrix_models['FN'] / (result_matrix_models['TP'] + result_matrix_models['FN'])
result_matrix_models['TNR'] = result_matrix_models['TN'] / (result_matrix_models['FP'] + result_matrix_models['TN'])

result_matrix_models['ACC'] = (result_matrix_models['TP'] + result_matrix_models['TN']) / \
                              (result_matrix_models['TP'] + result_matrix_models['FN'] + result_matrix_models['FP'] + result_matrix_models['TN'])
result_matrix_models['F1S'] = result_matrix_models['TP'] / \
                              (result_matrix_models['TP'] + 0.5 * (result_matrix_models['FP'] + result_matrix_models['FN']))

# Round all floats to 4 decimal places
# Note: round() does not work for floats with many decimal places
decimals = 4
for col in ['FPR', 'TPR', 'FNR', 'TNR', 'ACC', 'F1S']:
    result_matrix_models[col] = result_matrix_models[col].apply(lambda x: round(x, decimals))

# Move cols to end for similarity with ARIMA results
result_matrix_models = result_matrix_models[['ID', 'PARAMETER', 'MODEL', 'ENDOGENOUS', 'EXOGENOUS', 'FIRST_FORECAST',
                                             'ALARM_TYPE', 'FP', 'TP', 'FN', 'TN', 'FPR', 'TPR', 'FNR', 'TNR', 'ACC',
                                             'F1S', 'N_HIGH_ALARMS', 'N_LOW_ALARMS', 'N_CHUNKS', 'N_ITERATIONS']]

# Extract normal and scaled rows (see model number in comment below)
normal_model_numbers = [str(nr) for nr in list(range(20))][10:]
scaled_model_numbers = ['0' + str(nr) for nr in list(range(10))]
normal_rows, scaled_rows = list(), list()

"""
Normal:
    RNN,     MEDIAN:  10
    RNN,     MAX:     11 -> best for BP
    RNN,     MIN:     12 -> best for HR
    LSTM,    MEDIAN:  13
    LSTM,    MAX:     14
    LSTM,    MIN:     15
    GRU,     MEDIAN:  16
    GRU,     MAX:     17 -> best for O2
    GRU,     MIN:     18

Scaled:
    RNN,     MEDIAN:  01
    RNN,     MAX:     02
    RNN,     MIN:     03
    LSTM,    MEDIAN:  04
    LSTM,    MAX:     05
    LSTM,    MIN:     06
    GRU,     MEDIAN:  07
    GRU,     MAX:     08
    GRU,     MIN:     09
"""

for i, row in result_matrix_models.iterrows():
    if row['ID'].split('_')[2] in normal_model_numbers:
        normal_rows.append(row.values)
    elif row['ID'].split('_')[2] in scaled_model_numbers:
        scaled_rows.append(row.values)

# Add extracted rows to final matrices
result_matrix_models_normal = (pd.DataFrame(normal_rows, columns=result_matrix_models.columns)).reset_index(drop=True)
result_matrix_models_scaled = (pd.DataFrame(scaled_rows, columns=result_matrix_models.columns)).reset_index(drop=True)

# Show complemented result matrices per parameter
display(result_matrix_models_normal[result_matrix_models_normal['PARAMETER'] == 'HR'])
display(result_matrix_models_scaled[result_matrix_models_scaled['PARAMETER'] == 'HR'])

display(result_matrix_models_normal[result_matrix_models_normal['PARAMETER'] == 'BP'])
display(result_matrix_models_scaled[result_matrix_models_scaled['PARAMETER'] == 'BP'])

display(result_matrix_models_normal[result_matrix_models_normal['PARAMETER'] == 'O2'])
display(result_matrix_models_scaled[result_matrix_models_scaled['PARAMETER'] == 'O2'])

# Save result matrices as parquet
result_matrix_models_normal.to_parquet(f'../../data/darts/{n_chunks}_chunks/result_matrix_models_normal.parquet',
                                       engine='pyarrow')
result_matrix_models_scaled.to_parquet(f'../../data/darts/{n_chunks}_chunks/result_matrix_models_scaled.parquet',
                                       engine='pyarrow')

### Plot Accuracy, TPR, FNR, and TNR

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# "Group" result matrix by prefix of ID
plotdata = result_matrix_models.replace(['_H', '_L'], ['', ''], regex=True)

# Create subplots
sns.set_style('whitegrid')
fig, axs = plt.subplots(
    nrows=4,
    ncols=3,
    figsize=(15, 13),
    dpi=72
    )

plt.suptitle(f'Accuracy, TPR, FNR, and TNR of {n_chunks} Chunks', fontsize=22)

# Define y-limits
acc_ylimits = [0, max(result_matrix_models.ACC)]
tpr_ylimits = [0, max(result_matrix_models.TPR)]
fnr_ylimits = [0, max(result_matrix_models.FNR)]
tnr_ylimits = [0, max(result_matrix_models.TNR)]

# Actual plots
for i, parameter in enumerate(['HR', 'BP', 'O2']):
    sns.barplot(
        ax=axs[0, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='ACC',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None)
    axs[0, i].set_title(str(parameter), fontweight='bold', color= 'black', fontsize=20)
    axs[0, i].set_ylim(acc_ylimits)
    axs[0, i].set_xticklabels(axs[0, i].get_xticklabels(), rotation=90)

    sns.barplot(
        ax=axs[1, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='TPR',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None)
    axs[1, i].set_ylim(tpr_ylimits)
    axs[1, i].set_xticklabels(axs[1, i].get_xticklabels(), rotation=90)

    sns.barplot(
        ax=axs[2, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='FNR',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None)
    axs[2, i].set_ylim(fnr_ylimits)
    axs[2, i].set_xticklabels(axs[2, i].get_xticklabels(), rotation=90)

    sns.barplot(
        ax = axs[3, i],
        data = plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='TNR',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None)
    axs[3, i].set_ylim(tnr_ylimits)
    axs[3, i].set_xticklabels(axs[3, i].get_xticklabels(), rotation=90)

fig.tight_layout()
plt.show(fig)
#fig.savefig(f'../../plots/darts/{n_chunks}_chunks/tpr_fnr_tnr_acc_model_result_matrix.png', dpi=1200)

### Plot False Positive Ratio and F1 Score

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# "Group" result matrix by prefix of ID
plotdata = result_matrix_models.replace(['_H', '_L'], ['', ''], regex=True)

# Create subplots
sns.set_style('whitegrid')
fig, axs = plt.subplots(
    nrows=2,
    ncols=3,
    figsize=(15, 7),
    dpi=72
    )

plt.suptitle(f'FPR and F1S of {n_chunks} Chunks', fontsize=22)

# Define y-limits
fpr_ylimits = [0, max(result_matrix_models.FPR)]
f1s_ylimits = [0, max(result_matrix_models.F1S)]

# Actual plot
for i, parameter in enumerate(['HR', 'BP', 'O2']):

    sns.barplot(
        ax=axs[0, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='FPR',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None)
    axs[0, i].set_title(str(parameter), fontweight='bold', color= 'black', fontsize=14)
    axs[0, i].set_ylim(fpr_ylimits)
    axs[0, i].set_xticklabels(axs[0, i].get_xticklabels(), rotation=90)

    sns.barplot(
        ax=axs[1, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='F1S',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None)
    axs[1, i].set_ylim(f1s_ylimits)
    axs[1, i].set_xticklabels(axs[1, i].get_xticklabels(), rotation=90)

fig.tight_layout()
plt.show(fig)
#fig.savefig(f'../../plots/darts/{n_chunks}_chunks/fpr_f1s_model_result_matrix.png', dpi=1200)

## Analysis of Chunk-level Matrices

### Print One Chunk-level Matrix

In [ ]:
import pickle5 as pickle

# Define parameters to adjust
# Note: At the moment, the following analysis steps are only performed for one chunk-specific matrix file
n_chunks = 1000
model_type = 'LSTM'
parameter = 'o2'
endogenous_input = 'MEDIAN'
version = 'normal'
window_idx = 0

# Define path to all chunk-level matrices produced by prediction
path_to_chunk_matrices = f'../../data/darts/{n_chunks}_chunks'

# Read chunk-specific matrix
chunks_matrix_f = open(f'{path_to_chunk_matrices}/confusion_matrix_chunks_{model_type}_{parameter}_{endogenous_input}_'
                       f'{version}_window{window_idx}.pickle', 'rb')
chunks_matrix = pickle.load(chunks_matrix_f)
chunks_matrix_f.close()

# Show chunk-specific matrix
display(chunks_matrix)

### Add Metrics (and Other Remaining Columns) to Each Chunk

In [ ]:
import numpy as np

# Note: To avoid dividing by zero, zeros are converted to NaN before division (since any value divided by NaN gives NaN)

chunks_matrix['fp_tn_divisor'] = chunks_matrix['FP'] + chunks_matrix['TN']
chunks_matrix['TNR'] = chunks_matrix.TN.div(chunks_matrix.fp_tn_divisor.where(chunks_matrix.fp_tn_divisor != 0, np.nan))
chunks_matrix['FPR'] = chunks_matrix.FP.div(chunks_matrix.fp_tn_divisor.where(chunks_matrix.fp_tn_divisor != 0, np.nan)) # 1 - TNR

chunks_matrix['fn_tp_divisor'] = chunks_matrix['FN'] + chunks_matrix['TP']
chunks_matrix['TPR'] = chunks_matrix.TP.div(chunks_matrix.fn_tp_divisor.where(chunks_matrix.fn_tp_divisor != 0, np.nan))
chunks_matrix['FNR'] = chunks_matrix.FN.div(chunks_matrix.fn_tp_divisor.where(chunks_matrix.fn_tp_divisor != 0, np.nan)) # 1 - TPR

chunks_matrix['F1S_divisor'] = chunks_matrix['TP'] + 0.5 * (chunks_matrix['FP'] + chunks_matrix['FN'])
chunks_matrix['F1S'] = chunks_matrix.TP.div(chunks_matrix.F1S_divisor.where(chunks_matrix.F1S_divisor != 0, np.nan))

chunks_matrix['ACC_dividend'] = chunks_matrix['TN'] + chunks_matrix['TP']
chunks_matrix['ACC_divisor'] = chunks_matrix['fp_tn_divisor'] + chunks_matrix['fn_tp_divisor']
chunks_matrix['ACC'] = chunks_matrix.ACC_dividend.div(chunks_matrix.ACC_divisor.where(chunks_matrix.ACC_divisor != 0,
                                                                                      np.nan))

# Round all floats to 4 decimal places
# Note: round() does not work for floats with many decimal places
decimals = 4
for col in ['FPR', 'TPR', 'FNR', 'TNR', 'ACC', 'F1S']:
    chunks_matrix[col] = chunks_matrix[col].apply(lambda x: round(x, decimals))

# Sort and remove helper columns for similarity with model-level matrices
chunks_matrix = chunks_matrix[['CHUNK_ID', 'PARAMETER', 'MODEL', 'ENDOGENOUS', 'EXOGENOUS', 'FIRST_FORECAST',
                               'ALARM_TYPE', 'FP', 'TP', 'FN', 'TN', 'FPR', 'TPR', 'FNR', 'TNR', 'ACC', 'F1S',
                               'N_HIGH_ALARMS', 'N_LOW_ALARMS', 'N_ITERATIONS']]

# Show complemented chunk-level matrix for one chunk
display(chunks_matrix)

### Plot Correlation Between Chunk Length and F1 Score/ Specificity (TNR) of Chunk

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Define input length to adjust
input_length = 12

# Add column for chunk length to all chunks of matrix
chunks_matrix['LENGTH'] = chunks_matrix['N_ITERATIONS'] + input_length

for metric in ['F1S', 'TNR']:

    # Define background color, subplots and suptitle
    sns.set_style('whitegrid')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
    fig.suptitle(f'Correlation of Chunk Length and {metric} of Chunk', fontsize=14)

    if endogenous_input == 'MIN':
        ax1.set_visible(False)
        ax2.set_position([1, 0.2, 0.05, 0.2])
    else:
        # Extract chunks for high and low analysis plot
        high_chunks = chunks_matrix[chunks_matrix['ALARM_TYPE'] == 'High'][[metric, 'LENGTH']]

        # Introduce mean value for each length
        # Note: If mean value of metric is used, lines can be drawn again (with default of linestyle parameter)
        #high_chunks = high_chunks.astype(float)
        #high_chunks = high_chunks.groupby('LENGTH').mean()

        # Reset indices to make access via column names possible again
        high_chunks.reset_index(level=0, inplace=True, drop=True)

        # Add left plot (high threshold analysis)
        ax1.plot('LENGTH', metric, data=high_chunks, marker='o', color=sns.color_palette('colorblind')[0],
                 linestyle='None')
        ax1.set_title(f'{metric} Regarding High Thresholds', fontsize=10)
        ax1.set_xlabel('Chunk Length', fontsize=8)
        ax1.set_ylabel(f'{metric} of Chunk', fontsize=8)
        ax1.set_ylim(bottom=0, top=1.1)

    if endogenous_input == 'MAX':
        ax2.set_visible(False)
        ax1.set_position([0, 0.2, 0.05, 0.2])
    else:
        # Extract chunks for high and low analysis plot
        low_chunks = chunks_matrix[chunks_matrix['ALARM_TYPE'] == 'Low'][[metric, 'LENGTH']]

        # Introduce mean value for each length
        # Note: If mean value of metric is used, lines can be drawn again (with default of linestyle parameter)
        #low_chunks = vlow_chunks.astype(float)
        #low_chunks = low_chunks.groupby('LENGTH').mean()

        # Reset indices to make access via column names possible again
        low_chunks.reset_index(level=0, inplace=True, drop=True)

        # Add right plot (low threshold analysis)
        ax2.plot('LENGTH', metric, data=low_chunks, marker='o', color=sns.color_palette('colorblind')[1],
                 linestyle='None')
        ax2.set_title(f'{metric} Regarding Low Thresholds', fontsize=10)
        ax2.set_xlabel('Chunk Length', fontsize=8)
        ax2.set_ylabel(f'{metric} of Chunk', fontsize=8)
        ax2.set_ylim(bottom=0, top=1.1)

    # Improve layout and save figure
    fig.tight_layout()
    fig.show()
    #fig.savefig(f'../../plots/darts/{n_chunks}_chunks/correlation_chunk_length_and_{metric}_{model_type}_{parameter}_'
    #            f'{endogenous_input}_{version}.png', dpi=1200)

### Time-Series Plot of Chunk with Prediction

In [ ]:
print(f'Original amount of chunks: {len(chunks_matrix)}\n')

interesting_chunks = chunks_matrix[chunks_matrix.FPR.notnull() & chunks_matrix.F1S.notnull()]
print(f'Amount of interesting chunks: {len(interesting_chunks)}\n')

print(interesting_chunks[['CHUNK_ID', 'FPR', 'TPR', 'FNR', 'TNR', 'ACC', 'F1S', 'N_HIGH_ALARMS', 'N_LOW_ALARMS']])

In [ ]:
from darts import TimeSeries
import pandas as pd

chunks_ids_plotting = ['200098.0_220277.0_2136-03-27 12:00:00', '200061.0_220277.0_2134-01-24 14:15:00']

for chunk_id in chunks_ids_plotting:

    # Format chunk IDs into Windows format that have to be used when loading from or saving to Windows machine
    chunk_id_win10 = chunk_id.replace(':', '%3A')

    # Extract predicted series of chunk
    prediction_chunk_f = open(f'../../data/darts/{n_chunks}_chunks/{model_type}/{parameter}/{endogenous_input}/'
                              f'05_prediction_{chunk_id_win10}_{version}_window{window_idx}.pickle', 'rb')
    prediction_chunk = pickle.load(prediction_chunk_f)
    prediction_chunk_f.close()

    # Convert predicted series of chunk to TimeSeries object
    prediction_chunk = TimeSeries.from_dataframe(
        df=prediction_chunk,
        time_col='Time',
        value_cols=['Value'],
        freq='H')

    # Extract original series of chunk
    resampled_chunks = pd.read_parquet(f'../../data/resampling/resample_output_{parameter}_first{n_chunks}.parquet',
                                       engine='pyarrow')
    original_chunk = resampled_chunks[resampled_chunks['CHUNK_ID_FILLED_TH'] == chunk_id]

    # Convert original series of chunk to TimeSeries object
    original_chunk = TimeSeries.from_dataframe(
        df=original_chunk,
        time_col='CHARTTIME',
        value_cols=[f'VITAL_PARAMTER_VALUE_{endogenous_input}_RESAMPLING'],
        freq='H')

    # Actual plot
    sns.set_style('whitegrid')
    plt.figure(figsize=(8, 5))
    original_chunk.plot(label=f'{parameter.upper()} - actual')
    prediction_chunk.plot(label=f'{parameter.upper()} - predicted')

    # Adjust texts of plot
    plt.legend()
    plt.suptitle(f'Prediction of {parameter.upper()} with {n_chunks} Chunks, {endogenous_input} Input, and {model_type}'
                 f' Model', fontweight='bold')
    plt.xlabel('Time')
    plt.ylabel('Value')

    plt.show()
    #plt.savefig(f'../../plots/darts/{n_chunks}_chunks/prediction_{model_type}_{parameter}_{endogenous_input}_'
    #            f'{chunk_id_win10}_{version}.png', dpi=1200)